In [2]:
from transformers import RobertaForSequenceClassification, RobertaTokenizerFast, RobertaConfig
from datasets import load_dataset

model_name = 'microsoft/graphcodebert-base'

dataset_name = 'Reid996/big_clone_bench'



In [3]:
tokenizer = RobertaTokenizerFast.from_pretrained(model_name)
dataset = load_dataset(dataset_name)

/home/zixian/.conda/envs/LLM_code_clone_validation/lib/python3.8/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/772 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/539 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at microsoft/graphcodebert-base and are newly initialized: ['classifier.dense.bias', 'classifier.out_proj.bias', 'classifier.out_proj.weight', 'classifier.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/home/zixian/.conda/envs/LLM_code_clone_validation/lib/python3.8/site-packages/datasets/table.py:1421: FutureWarning: promote has been superseded by promote_options='default'.
  table = cls._concat_blocks(blocks, axis=0)


In [5]:
MAX_LENGTH = 255
def tokenization(row):
    tokens_input = tokenizer([row['func1'], row['func2']], padding='max_length', truncation=True, return_tensors='pt',max_length=MAX_LENGTH)
    tokens_input['input_ids'] = tokens_input['input_ids'].flatten()
    tokens_input['attention_mask'] = tokens_input['attention_mask'].flatten()
    return tokens_input

In [6]:
dataset = dataset.map(tokenization)

Map:   0%|          | 0/901376 [00:00<?, ? examples/s]

Map:   0%|          | 0/415434 [00:00<?, ? examples/s]

Map:   0%|          | 0/415444 [00:00<?, ? examples/s]

In [13]:
import torch
import evaluate
import numpy as np
from transformers import TrainingArguments, Trainer, RobertaConfig
dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])

dataset_train = dataset['train']
dataset_test = dataset['test']
dataset_val = dataset['validation']

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

config = RobertaConfig.from_pretrained(model_name, num_labels=2)
model = RobertaForSequenceClassification.from_pretrained(model_name, config=config).to(device)

accuracy = evaluate.load("accuracy")
recall = evaluate.load("recall")
precision = evaluate.load("precision")
f1 = evaluate.load("f1")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return {"accuracy": accuracy.compute(predictions=predictions, references=labels)["accuracy"], "precision": precision.compute(predictions=predictions, references=labels)["precision"], "recall": recall.compute(predictions=predictions, references=labels)["recall"], "f1": f1.compute(predictions=predictions, references=labels)["f1"]}

BATCH_SIZE = 16
STEPS = 32
training_args = TrainingArguments(
    report_to= 'wandb',
    run_name='tmp1',
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    learning_rate=2e-5,             # Learning rate
    adam_epsilon=1e-8,              # Epsilon for Adam optimizer
    num_train_epochs= 2,             # Total number of training epochs
    logging_dir='./logs',           # Directory for storing logs
    logging_steps=STEPS,
    evaluation_strategy="epoch",
    output_dir ="./output",
    dataloader_pin_memory=True,
    dataloader_num_workers=4, # how many cpus to use to load the data while training
    do_eval=True,                 # Perform evaluation at the end of training
    save_strategy="epoch",
    load_best_model_at_end = True,
    metric_for_best_model = 'f1',
    save_total_limit=3
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset_train,
    eval_dataset=dataset_val,      # Evaluation dataset
    compute_metrics=compute_metrics,
)

trainer.train()


/home/zixian/.conda/envs/LLM_code_clone_validation/lib/python3.8/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at microsoft/graphcodebert-base and are newly initialized: ['classifier.dense.bias', 'classifier.out_proj.bias', 'classifier.out_proj.weight', 'classifier.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PAR

Epoch,Training Loss,Validation Loss


Epoch,Training Loss,Validation Loss


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.000000,0.214025,0.974562,0.893158,0.912992,0.902966


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.000000,0.214025,0.974562,0.893158,0.912992,0.902966


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.000000,0.214025,0.974562,0.893158,0.912992,0.902966
2,0.000000,0.266581,0.973868,0.912448,0.883172,0.897571


TrainOutput(global_step=112672, training_loss=0.013599242318379626, metrics={'train_runtime': 96300.8187, 'train_samples_per_second': 18.72, 'train_steps_per_second': 1.17, 'total_flos': 4.724711532205056e+17, 'train_loss': 0.013599242318379626, 'epoch': 2.0})